# Usage

In [ ]:
from hydrodata import Station
import hydrodata.datasets as hds
from hydrodata import plot

Hydrodata module downloads daily climate data for a USGS station based its ID or coordinates (longitude and latitude). It requires at least three parameters: start date, end date and USGS station ID or coordinates. The data are downloaded from the [Daymet](https://daymet.ornl.gov/) and [NWIS](https://nwis.waterdata.usgs.gov/nwis) datasets.

## Climate Data

For example, let's consider the following USGS station:

In [ ]:
station_id = '01031500'
start, end = '2000-01-01', '2010-01-21'

Now, we can define an instance based on the station coordinate. Upon instantiation, the station and its watershed characteristics are found using [NLDI](https://labs.waterdata.usgs.gov/about-nldi/) service.

In [ ]:
piscataquis = Station(start, end, station_id=station_id, data_dir="examples/data")

We the retrieved information such as the watershed geometry we can use the `datasets` module to access other databases. For example, we can find the USGS station upstream of the main river channel up to a certain distance, say 150 km. Also, all the USGS stations inside the watershed can be found.

In [ ]:
stations = piscataquis.watershed.get_stations()
stations_upto_150 = piscataquis.watershed.get_stations(navigation="upstreamMain", distance=150)

ax = piscataquis.watershed.basin.plot(color='white', edgecolor='black', zorder=1, figsize = (10, 10))
piscataquis.tributaries.plot(ax=ax, label='Tributaries', zorder=2)
piscataquis.main_channel.plot(ax=ax, color='green', lw=3, label='Main', zorder=3)
stations.plot(ax=ax, color='black', label='All stations', marker='s', zorder=4)
stations_upto_150.plot(ax=ax, color='red', label='Stations up to 150 km upstream of main', marker='*', zorder=5)
ax.legend(loc='best');

Next, let's get the Digital Elevation Model of the watershed from [OpenTopography](https://opentopography.org/) service.

In [ ]:
piscataquis_dem = hds.dem_bygeom(piscataquis.geometry)
piscataquis_dem.plot(size=8);

The climate and streamflow observations data can be retrieved for the location of interest:

In [ ]:
piscataquis_clm = hds.deymet_byloc(piscataquis.lon, piscataquis.lat, start=piscataquis.start, end=piscataquis.end)
piscataquis_clm['Q (cms)'] = hds.nwis(piscataquis.station_id, piscataquis.start, piscataquis.end)
piscataquis_clm.head()

In addition to point-based data, gridded data can also be accessed. For example, let's get the climate data for the whole watershed. The `daymet_bygeom` function can also compute Potential EvapoTranspiration using FAO-56.

In [ ]:
variables = ["tmin", "tmax", "prcp"]
piscataquis_grd = hds.daymet_bygeom(piscataquis.geometry, start='2005-01-01', end='2005-01-31', variables=variables, pet=True)

The data is returned as an `xarray` dataset which provides access to many exploratory features. It's noteworthy that Daymet data projection is Lambert Conformal thus for plotting purposes it should be considered.

In [ ]:
import cartopy.crs as ccrs
import matplotlib.pyplot as plt

# Daymet data projection
lcc = ccrs.LambertConformal(standard_parallels=(25, 60), central_longitude=-100, central_latitude=42.5)
fig = plt.figure(figsize=(12, 8))
ax = plt.axes(projection=ccrs.PlateCarree())
cax = fig.add_axes([ax.get_position().x1 + 0.14, ax.get_position().y0, 0.02, ax.get_position().height])
piscataquis_grd.isel(time=1).tmin.plot(ax=ax, transform=lcc, cbar_kwargs={"cax":cax});

Access to actual EvapoTranspiration data through [SSEBop](https://earlywarning.usgs.gov/ssebop/modis/daily) is also possible. However, since there's still no web service available for subsetting the data, `ssebopeta_bygeom` first downloads the data for the requested period then subsets the data based on the provided geometry locally. Therefore, it's not as fast as other function and the bottleneck is download speed which could vary depending on the available internet speed.

In [ ]:
piscataquis_eta = hds.ssebopeta_bygeom(piscataquis.geometry, start='2005-01-01', end='2005-01-31')
piscataquis_eta.isel(time=4).eta.plot(size=8);

Another database that is accessible is land use, land cover data though Multi-Resolution Land Characteristics (MRLC) Consortium. By default the data are downloaded from NLCD 2016.

In [ ]:
piscataquis_lulc = hds.NLCD(piscataquis.geometry,
                            years={'impervious': 2016, 'cover': 2016, 'canopy': 2016},
                            data_dir=f"examples/data/{piscataquis.station_id}")

In addition to the downloaded raster files, some statistics on the data is performed and the result is returned as a dictionary. For example, let's get the percentages of each cover category.

In [ ]:
piscataquis_lulc['cover']['categories']

The downloaded raster files can be plotted using [rasterio](https://rasterio.readthedocs.io/en/latest/). Hydrodata also provides a helper function for getting the official [legends](https://www.mrlc.gov/data/legends/national-land-cover-database-2016-nlcd2016-legend) for the land cover dataset.

In [ ]:
import rasterio
from rasterio.plot import show

cmap, norm = plot.cover_lenegeds()

rs = [rasterio.open(f"examples/data/{piscataquis.station_id}/{d}_2016.geotiff") for d in ['canopy', 'cover', 'impervious']]
fig, (ax1, ax2, ax3) = plt.subplots(1, 3, figsize=(21, 7), dpi=300)
show(rs[0], ax=ax1, title=f'Canopy 2011', cmap='Greens')
show(rs[1], ax=ax2, title=f'Cover 2011', cmap=cmap, norm=norm)
show(rs[2], ax=ax3, title=f'Impervious 2011');

Moreover, longitude and latitude coordinate can also be used for downloading the data. The coordinates doesn't have to be exact since the closest station to the specified coordinates is found automatically. For example, let's find the climate data for a station close to the following coordinates:

In [ ]:
lon, lat = -75.097, 40.016
frankford = Station(start, end, coords=(lon, lat), data_dir="examples/data")
frankford_Q = hds.nwis(frankford.station_id, frankford.start, frankford.end)

## Plotting

The hydrologic signatures can be plotted easily using the `signatures` function from the `plot` module. This functions convert the streamflow data from cubic meter per second to millimeter per day based on the watershed area which should be provided in km$^2$.

In [ ]:
plot.signatures({'Piscataquis': (piscataquis_clm['Q (cms)'], piscataquis.drainage_area)},
                prcp=piscataquis_clm['prcp (mm/day)'],
                title=piscataquis.name)

### Observed discharge for multiple stations

The `plot_discharge` function can be used to compare hydrological signatures of multiple daily discharges time series of multiple stations. This plot doesn't include the precipitation data and only compares the discharges. Let's compare the Frankford and Fishing watershed. The discharge dictionary should be provided as before.

In [ ]:
Q_daily = {'Piscataquis': (piscataquis_clm['Q (cms)'], piscataquis.drainage_area),
           'Frankford': (frankford_Q, frankford.drainage_area)}
plot.signatures(Q_daily=Q_daily,
                title='Streamflow data for two watersheds')